# Találjunk ki egy számot: lineáris regresszió

## Használjunk több változót

In [1]:
from sklearn.datasets import fetch_california_housing
import pandas as pd

def kaliforniai_lakásadatok_betöltése():
    adathalmaz = fetch_california_housing()
    X = pd.DataFrame(data=adathalmaz.data,
                     columns=adathalmaz.feature_names)
    y = pd.Series(data=adathalmaz.target, name="cél")
    return X, y

X, y = kaliforniai_lakásadatok_betöltése()

In [2]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

regresszió = Pipeline(steps=[
    ('skálázó', StandardScaler()),
    ('modell', LinearRegression())])

regresszió.fit(X, y)

Pipeline(steps=[('skálázó', StandardScaler()), ('modell', LinearRegression())])

In [3]:
pontszám = regresszió.score(X, y)
print(f"{pontszám:.3f}")

0.606


In [4]:
for jellemző, együttható in zip(X.columns,
                        regresszió['modell'].coef_):
    print(f"{jellemző:12}: {együttható:>7.3f}")

MedInc      :   0.830
HouseAge    :   0.119
AveRooms    :  -0.266
AveBedrms   :   0.306
Population  :  -0.005
AveOccup    :  -0.039
Latitude    :  -0.900
Longitude   :  -0.871


# Térjünk át a logisztikus regresszióra

## Alkalmazzunk logisztikus regressziót

In [5]:
import pandas as pd

def palmer_pingvinek_betöltése(only_numeric=True,
                         no_missing=True,
                         multiclass=True):
    url = "https://raw.githubusercontent.com/"
    url += "allisonhorst/palmerpenguins/main/"
    url += "inst/extdata/penguins.csv"
    numerikus_jellemzők = ["bill_length_mm",
                        "bill_depth_mm",
                        "flipper_length_mm",
                        "body_mass_g"]
    kategorikus_jellemzők = ["island", "sex"]
    adatok = pd.read_csv(url)
    if no_missing:
        adatok = adatok.dropna()
    if multiclass:
        cél = adatok.species.replace({'Adelie':1,
                                       'Gentoo':2,
                                       'Chinstrap':3})
    else:
        cél = adatok.species.replace({'Adelie':1,
                                       'Gentoo':0,
                                       'Chinstrap':0})
    if only_numeric:
        return adatok[numerikus_jellemzők], cél
    else:
        return adatok[numerikus_jellemzők +
                    kategorikus_jellemzők], cél
    
X, y = palmer_pingvinek_betöltése(only_numeric=True,
                            no_missing=True,
                            multiclass=False)

In [6]:
from sklearn.linear_model import LogisticRegression

logisztikus = Pipeline(steps=[
    ('skálázó', StandardScaler()),
    ('modell', LogisticRegression())])
logisztikus.fit(X.iloc[:-1], y.iloc[:-1])

kihagyott_sor = X.iloc[[-1]]
előrejelzés = logisztikus.predict(kihagyott_sor)
valség = logisztikus.predict_proba(kihagyott_sor)
print (f"Előrejelzett osztály: {előrejelzés[0]}, tényleges osztály: " +
       f"{y.iloc[-1]}")
print (f"{valség[0, 0]:.3f} valószínűséggel")

Előrejelzett osztály: 0, tényleges osztály: 0
0.987 valószínűséggel


## Tekintsük azt az esetet, amikor több osztály van

In [7]:
from sklearn.model_selection import train_test_split

X, y = palmer_pingvinek_betöltése(only_numeric=True,
                            no_missing=True,
                            multiclass=True)
X_tanító, X_teszt, y_tanító, y_teszt = train_test_split(
    X, y, test_size=0.33, random_state=42)

In [8]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multiclass import OneVsOneClassifier
eat = OneVsRestClassifier(logisztikus).fit(X_tanító, y_tanító)
eae = OneVsOneClassifier(logisztikus).fit(X_tanító, y_tanító)
print('Egy a többi ellen pontossága: %.3f' % eat.score(
    X_teszt, y_teszt))
print('Egy az egy ellen pontossága: %.3f' % eae.score(
    X_teszt, y_teszt))

Egy a többi ellen pontossága: 0.973
Egy az egy ellen pontossága: 0.982


# Egyszerűsítsük le a dolgokat annyira, mint a naiv Bayes-algoritmus

## Jelezzük előre szövegek osztályozását

In [9]:
import numpy as np
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text \
    import CountVectorizer
import sklearn.feature_extraction.text as szöveg
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.metrics import accuracy_score

hírcsoportok_tanító = fetch_20newsgroups(
    subset='train', remove=('headers', 'footers',
                            'quotes'))
hírcsoportok_teszt = fetch_20newsgroups(
    subset='test', remove=('headers', 'footers',
                           'quotes'))

In [10]:
bernoulli_nb = BernoulliNB(alpha=0.01)
multinomiális_nb = MultinomialNB(alpha=0.01)

multinomiális_vektorizáló = CountVectorizer(
    stop_words='english', binary=False)
bináris_vektorizáló = CountVectorizer(
    stop_words='english', binary=True)

In [11]:
tanító_célok = hírcsoportok_tanító.target
teszt_célok = hírcsoportok_teszt.target

multinomiális_X = np.abs(
    multinomiális_vektorizáló.fit_transform(
        hírcsoportok_tanító.data))
multinomiális_Xt = np.abs(
    multinomiális_vektorizáló.transform(
        hírcsoportok_teszt.data))
bináris_X = bináris_vektorizáló.fit_transform(
    hírcsoportok_tanító.data)
bináris_Xt = bináris_vektorizáló.transform(
    hírcsoportok_teszt.data)

In [12]:
multinomiális_nb.fit(multinomiális_X, tanító_célok)
bernoulli_nb.fit(bináris_X, tanító_célok)

for name, model, data in [
    ('BernoulliNB', bernoulli_nb, bináris_Xt),
    ('MultinomialNB', multinomiális_nb, multinomiális_Xt)]:
    pontosság = accuracy_score(
        y_true=teszt_célok, y_pred=model.predict(data))
    print(f"A {name} pontossága: {pontosság:.3f}")

A BernoulliNB pontossága: 0.567
A MultinomialNB pontossága: 0.653


In [13]:
print(f'tanító bejegyzések: {len(hírcsoportok_tanító.data)}')
D = {szó: True for post in hírcsoportok_tanító.data
     for szó in post.split(' ')}
print(f'tanító szavak: {len(D)}')
print(f'tesztbejegyzések: {len(hírcsoportok_teszt.data)}')

tanító bejegyzések: 11314
tanító szavak: 300972
tesztbejegyzések: 7532


# Használjunk lusta tanulást a legközelebbi szomszédokkal

## Adjunk előrejelzést a szomszédok megfigyelése után

In [14]:
import matplotlib.pyplot as dg
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

számjegyek = load_digits()
X_tanító, X_teszt, y_tanító, y_teszt = train_test_split(
    számjegyek.data, számjegyek.target,
    test_size=0.33, random_state=42)

In [15]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5, p=2)
knn.fit(X_tanító, y_tanító)

KNeighborsClassifier()

In [16]:
print('Pontosság: %.3f' % knn.score(X_teszt, y_teszt) )
print(f"Előrejelzés: {knn.predict(X_teszt[-15:,:])}")
print(f"Tényleges:     {y_teszt[-15:]}")

Pontosság: 0.993
Előrejelzés: [2 1 1 2 2 4 8 7 5 8 8 9 4 9 0]
Tényleges:     [2 1 1 2 2 4 8 7 5 8 8 9 4 9 0]


## Válasszuk meg okosan a k paramétert

In [17]:
for k in [1, 3, 5, 7, 10, 50, 100]:
    kNN = KNeighborsClassifier(n_neighbors=k)
    kNN.fit(X_tanító, y_tanító)
    teszt_pontszám = kNN.score(X_teszt, y_teszt)
    print(f"k= {k:3} \t pontosság= {teszt_pontszám:.3f}")

k=   1 	 pontosság= 0.985
k=   3 	 pontosság= 0.990
k=   5 	 pontosság= 0.993
k=   7 	 pontosság= 0.990
k=  10 	 pontosság= 0.983
k=  50 	 pontosság= 0.929
k= 100 	 pontosság= 0.899
